In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [2]:
df1=pd.read_csv('preprocess.csv')
df2=pd.read_csv('Testpreprocess.csv')

In [3]:
df1.shape

(630000, 13)

In [4]:
df2.shape

(270000, 12)

In [5]:
# Drop rows (Train data) where 'n_votes' is <1
df1 = df1.loc[df1['n_votes']>0]
# Display the resulting DataFrame
df1.shape

(263685, 13)

In [6]:
columns_to_drop = ['user_id', 'book_id','review_id','review_text','date_added','date_updated','read_at','started_at','n_votes','n_comments','review']
df1=df1.drop(columns=columns_to_drop)
df1.shape

(263685, 2)

In [7]:
unique_values = df1['rating'].value_counts()
print(unique_values)

rating
4    90455
5    89739
3    45945
2    18999
0     9494
1     9053
Name: count, dtype: int64


In [8]:
columns_to_drop = ['user_id', 'book_id','review_text','date_added','date_updated','read_at','started_at','n_votes','n_comments','review']
df2=df2.drop(columns=columns_to_drop)
df2.shape

(270000, 2)

In [9]:
df2.head(5)

,review_id,destr_review
0,b3d1248da3dd5cb0c15d5e737793ca32,tw book view spoiler wicca witchcraft minor ad...
1,8ac45a1ad472e40dd19245317087f009,found like half extremely interesting excellen...
2,e083ab7283f9c8f0edd6f5673af348a9,dragon path sneak start book pause middle keep...
3,167e419508a6a188472e9568226da48a,star well written drawn story never liked jona...
4,03a8a7306e3ef4c1ffd95de068b58b4e,crestfallen feel read book hidden sound awesom...


In [10]:
col=['destr_review']
df3=df2.drop(columns=col)

In [11]:
col1=['review_id']
df2=df2.drop(columns=col1)

In [12]:
df1 = df1.groupby('rating', group_keys=False).apply(lambda x: x.head(9053))
df1.shape

(54318, 2)

In [31]:
#Vectorization
# max_features = 60500
# df1['destr_review'] = df1['destr_review'].fillna('')
# tf_idf_vect1=TfidfVectorizer(ngram_range=(1,2),max_features=max_features)
# X_tfidf1=tf_idf_vect1.fit_transform((df1['destr_review']).values)

In [32]:
# df2['destr_review'] = df2['destr_review'].fillna('')
# X_tfidf2=tf_idf_vect1.transform((df2['destr_review']).values)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
df1['destr_review'] = df1['destr_review'].fillna('')
vectorizer = CountVectorizer(ngram_range=(1,2))
X_tfidf1=vectorizer.fit_transform(((df1['destr_review']).values))

In [14]:
df2['destr_review'] = df2['destr_review'].fillna('')
X_tfidf2=vectorizer.transform((df2['destr_review']).values)

In [15]:
X_tfidf1.shape

(54318, 2714637)

In [16]:
X_tfidf2.shape

(270000, 2714637)

In [17]:
labels = df1['rating']

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf1, labels, test_size=0.2, random_state=0)

In [20]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('rf', RandomForestClassifier())
])


In [24]:
param_grid = {
    'tfidf__max_features': [1000, 5000, None],
    'rf__n_estimators': [50, 100, 200],
    'rf__max_depth': [None, 10, 20],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4]
}


In [26]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD  # Use TruncatedSVD for dimensionality reduction

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(lowercase=False, stop_words='english', max_features=5000)),
    ('reduce_dim', TruncatedSVD(n_components=100)),  # Dimensionality reduction for sparse matrices
    ('rf', RandomForestClassifier())
])


In [31]:
best_params = grid_search.cv_results_['params'][grid_search.best_index_]
print("Best parameters found: ", best_params)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))
print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'

In [ ]:
y_pred = model.predict(X_test)

In [24]:
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.47
